In [1]:
import librosa
import torch
from src.Dataset import AudioDataset
from torch.utils.data import DataLoader


In [2]:

# Load dataset
dataset = AudioDataset()

# Create test dataloader
test_loader = DataLoader(
    dataset, 
    batch_size=10, 
    shuffle=True, 
    num_workers=4
)


Dataset initialized with 30000 files
Using data directory: d:\Users\kaasa\Documents\DriveBackup\GitHub\Audio-MNIST-Generation\Data
Device: cuda


In [3]:
import os
model = torch.load(
    os.path.join("results", 'vqvae_run_20241216_143233', 'final_model.pt')
)
model.eval()
model = model.to('cuda')

C:\Users\kaasa\AppData\Local\Temp\ipykernel_11748\342511904.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(


In [5]:
input = torch.randn(32,1,128,94)
x_recon, ze, zq, indices, KL_loss, commit_loss = model(input.to('cuda'))

In [6]:
ze.shape

torch.Size([32, 32, 32, 23])

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from src.Transformer import TransformerModel
import matplotlib.pyplot as plt
from src.utils import *
from tqdm import tqdm
import os
from src.Dataset import AudioDataset

# Update block_size to accommodate label + flattened indices
block_size = 737  # 1 label + 736 flattened indices
n_embd = 512  # Adjust embedding dimension if needed
vocab_size = 512  # Number of unique indices in codebook
device = 'cuda'
# Load VQVAE model
vqvae_model = torch.load(
    os.path.join("results", 'vqvae_run_20241216_143233', 'final_model.pt'), 
    weights_only=False
).to(device)

# Load dataset
dataset = AudioDataset()

# Split dataset
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    persistent_workers=True
)


Dataset initialized with 30000 files
Using data directory: d:\Users\kaasa\Documents\DriveBackup\GitHub\Audio-MNIST-Generation\Data
Device: cuda


In [12]:

# Initialize Transformer Model 
m = TransformerModel().to(device)

# Load existing transformer model if available
try:
    m = torch.load(TRANSFORMER_MODEL_PATH, weights_only=False).to(device)
    print("Loaded existing transformer model")
except:
    print("Creating new transformer model")
    
# Optimizer
epochs = 10
learning_rate = 1e-4
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)


def prepare_transformer_data(vqvae_model, dataloader, block_size=737):
    """
    Prepare data for transformer by encoding spectrograms and flattening indices
    
    Args:
        vqvae_model (VQVAE): Trained VQVAE model
        dataloader (DataLoader): DataLoader containing spectrograms and labels
        block_size (int): Total sequence length including label
    
    Returns:
        tuple: Prepared x and y data for transformer training
    """
    # Prepare data
    data_label = []
    for x, label in dataloader:
        # Encode spectrogram to indices
        x = x.to(device).unsqueeze(1)
        #print(x.shape)
        # Get indices from encoder
        ze = vqvae_model.Encoder(x)
        
        # Quantize indices 
        indices = vqvae_model.codebook.quantize_indices(ze)
        
        # Flatten indices
        flattened_indices = indices.flatten(start_dim=1)
        
        # Combine label with flattened indices
        combined = torch.cat([label.unsqueeze(1).to(device), flattened_indices], dim=1)
        data_label.append(combined)
        #print(combined.shape)
    # Concatenate all data
    data_label = torch.cat(data_label, dim=0)
    
    # Prepare sequences
    x = torch.stack([data_label[i,:block_size-1] for i in range(len(data_label))])
    y = torch.stack([data_label[i,1:block_size+1] for i in range(len(data_label))])
    print(x.shape)
    print(y.shape)
    return x, y

Initialized transformer


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:

# Prepare training and test data
x_train, y_train = prepare_transformer_data(vqvae_model, train_loader)
x_test, y_test = prepare_transformer_data(vqvae_model, test_loader)

# Prepare data loaders
x_train_loader = DataLoader(x_train, batch_size=32)
y_train_loader = DataLoader(y_train, batch_size=32)
x_test_loader = DataLoader(x_test, batch_size=32)
y_test_loader = DataLoader(y_test, batch_size=32)

print("started training")
# Training parameters






torch.Size([21000, 736])
torch.Size([21000, 736])
torch.Size([4500, 736])
torch.Size([4500, 736])
started training


In [11]:

def train_model(m, x_train_loader, y_train_loader, x_test_loader, y_test_loader, epochs=10):
    train_loss = []
    validation_loss = []
    for iter in tqdm(range(epochs)):
        m.train()
        total_loss = 0
        for xb, yb in zip(x_train_loader, y_train_loader):
            xb, yb = xb.to(device), yb.to(device)
            print(xb.shape,yb.shape)
            logits, loss = m(xb, yb)
            optimizer.zero_grad(set_to_none=True)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_loss.append(total_loss / len(x_train_loader))
        print(f'Epoch {iter + 1}, Train Loss: {total_loss / len(x_train_loader)}')
        
        m.eval()
        total_loss = 0
        for xb, yb in zip(x_test_loader, y_test_loader):
            xb, yb = xb.to(device), yb.to(device)
            logits, loss = m(xb, yb)
            total_loss += loss.item()
        validation_loss.append(total_loss / len(x_test_loader))
        print(f'Epoch {iter + 1}, Validation Loss: {total_loss / len(x_test_loader)}')
    
    plt.figure(figsize=(10, 5)) 
    plt.plot(range(1,epochs+1), train_loss, 'bo-', label='Training Loss') 
    plt.plot(range(1,epochs+1), validation_loss, 'ro-', label='Validation Loss') 
    plt.xlabel('Epochs') 
    plt.ylabel('Loss') 
    plt.title('Training and Validation Loss over Epochs') 
    plt.legend() 
    plt.grid(True) 
    plt.savefig("TransformerLossCurves.png")
    return m

# Train the model
m = train_model(m, x_train_loader, y_train_loader, x_test_loader, y_test_loader, epochs)

# Save the trained model
torch.save(m, TRANSFORMER_MODEL_PATH)

# Example generation
context = torch.tensor([[3],[42],[62]]).to(device=device)
print(m.generate(context, max_new_tokens=736))

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
